In [ ]:
# load requirments library
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

import math
import time
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import confusion_matrix
 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import variable

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

#our libraries
#from train import train_model
#from model_utils import *
#from predict_utils import *
#from vis_utils import *

# some initial setup
np.set_printoptions(precision = 2)
use_gpu = torch.cuda.is_available()
np.random.seed(1234)



import pandas as pd


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')




DATA_DIR = "/content/gdrive/My Drive/data/dataset"

sz = 224
batch_size = 16

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
##%%%%dataset
DATA_DIR = "/content/gdrive/My Drive/data/dataset"

#!ls"/content/drive/My Drive/ColabNotebooks/data"
#DATA_DIR = "https://drive.google.com/drive/folders/136QdGHviw2R2yt3S4dE7COICoyEDhLeI?usp=sharing"
#os.listdir(DATA_DIR)
###from google.colab import files
###uploaded = files.upload()



In [ ]:

os.listdir(DATA_DIR)



['valid', 'train', 'Test']

In [ ]:
trn_dir = f'{DATA_DIR}train'
valid_dir = f'{DATA_DIR}valid'

#os.listdir(trn_dir)


In [ ]:
trn_dir

'/content/gdrive/My Drive/data/datasettrain'

In [ ]:
os.listdir(trn_dir)

FileNotFoundError: ignored

In [ ]:
#show image
trn_fnames = glob.glob(f'{trn_dir}/*/*.jpg')
trn_fnames[:5]


In [ ]:
#dir(glob)

In [ ]:
#plot image
img = plt.imread(trn_fnames[1])
plt.imshow(img)

In [ ]:
##datasets
train_ds = datasets.ImageFolder(trn_dir)
train_ds.classes


In [ ]:
#%%for indexing classes to 0 & 1
train_ds.class_to_idx
##for watching path
train_ds.root 
# for seeing images with classes
train_ds.imgs



In [ ]:
##%%Transformations; data loader object uses theses transformations when loading data.  
tfms = transforms.Compose([
                           transforms.Resize((sz,sz)), #PIL Image
                           transforms.ToTensor(),  #change to matrix
                           transforms.Normalize([0.485,0.456, 0.406], [0.229, 0.224,0.225])    
])

train_ds = datasets.ImageFolder(trn_dir,transform = tfms)
valid_ds = datasets.ImageFolder(val_dir,transform = tfms)



In [ ]:
##dataloaders
train_dl = torch.utils.data.DataLoader(train_ds,batch_size=batch_size, shuffle = True, num_workers = 8)
valid_dl = torch.utils.data.DataLoader(valid_ds,batch_size=batch_size, shuffle = True, num_workers = 8)



In [ ]:
##iterator
inputs, targets = next(iter(train_dl))
out = torchvision.utils.make_grid(inputs, padding=3)


In [ ]:
##ModelCNN
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()

    self.conv1 = nn.Sequential(
        nn.conv2d(3, 16, kernel_size=5, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2)        
    )

    self.conv2 = nn.Sequential(
        nn.conv2d(16, 32, kernel_size = 5, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(56 * 56 * 32, 2)

def forward(self, x):
  out = self.conv1(x)
  out = self.conv2(out)
  out = out.view(out.size(0), -1) ###for changing 4 to 2 tensor
  out = self.fc(out)
  return out


In [ ]:
#now it's time to create model
model = SimpleCNN()


In [ ]:
##lossfunction and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parametes(), lr=0.002, momentum=0.9)

In [ ]:
##train
num_epochs = 10
losses = []
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_dl):
        inputs = to_var(inputs)
        targets = to_var(targets)
        
        # forwad pass
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # loss
        loss = criterion(outputs, targets)
        losses += [loss.data[0]]
        # backward pass
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # report
        if (i + 1) % 50 == 0:
            print('Epoch [%2d/%2d], Step [%3d/%3d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_ds) // batch_size, loss.data[0]))


